In [1]:
#Read in libraries/extensions
import arcpy
arcpy.CheckOutExtension("ImageAnalyst")
arcpy.env.overwriteOutput = True
from arcpy.ia import *
from arcpy import env
from arcpy.sa import *
import os

In [2]:
#Define input folders
in_fold = "C:/Maxwell_Data/topo_data/topo_dl_data/topo_dl_data/"
in_toposF = "ky_topos/"
in_minesF = "ky_mines/"
in_quadsF = "ky_quads/"
chips_outF = in_fold + "processing/ky_chips4/"
to_8bitF = in_fold + "processing/ky_topo8bit4/"

#Create new directories
os.mkdir(chips_outF)
os.mkdir(to_8bitF)

#List all topo maps
arcpy.env.workspace =  in_fold + in_toposF
chips = arcpy.ListRasters()
print(chips)

['KY_Adams_708051_1971_24000_geo.tif', 'KY_Ano_708081_1979_24000_geo.tif', 'KY_Argillite_708084_1972_24000_geo.tif', 'KY_Artemus_708079_1974_24000_geo.tif', 'KY_Ault_708101_1962_24000_geo.tif', 'KY_Balkan_803302_1974_24000_geo.tif', 'KY_Barbourville_708122_1974_24000_geo.tif', 'KY_Barcreek_708125_1979_24000_geo.tif', 'KY_Barthell_803308_1954_24000_geo.tif', 'KY_Beattyville_708136_1961_24000_geo.tif', 'KY_Belfry_708145_1992_24000_geo.tif', 'KY_Belfry_708147_1978_24000_geo.tif', 'KY_Bernstadt_708158_1969_24000_geo.tif', 'KY_Beverly_708161_1979_24000_geo.tif', 'KY_Billows_708177_1952_24000_geo.tif', 'KY_Blackey_708183_1954_24000_geo.tif', 'KY_Blackwater_708188_1979_24000_geo.tif', 'KY_Blaine_708191_1953_24000_geo.tif', 'KY_Bledsoe_708195_1980_24000_geo.tif', 'KY_Boltsfork_708203_1971_24000_geo.tif', 'KY_Booneville_708207_1961_24000_geo.tif', 'KY_Booneville_708209_1961_24000_geo.tif', 'KY_Brushart_708255_1949_24000_geo.tif', 'KY_Buckhorn_708264_1961_24000_geo.tif', 'KY_Buckhorn_708265_1979

In [3]:
#Make list and remove file extensions
chip_n = list()
for c in chips:
    c1 = os.path.basename(c)
    c2 = os.path.splitext(c1)[0]
    chip_n.append(c2)
print(chip_n)

['KY_Adams_708051_1971_24000_geo', 'KY_Ano_708081_1979_24000_geo', 'KY_Argillite_708084_1972_24000_geo', 'KY_Artemus_708079_1974_24000_geo', 'KY_Ault_708101_1962_24000_geo', 'KY_Balkan_803302_1974_24000_geo', 'KY_Barbourville_708122_1974_24000_geo', 'KY_Barcreek_708125_1979_24000_geo', 'KY_Barthell_803308_1954_24000_geo', 'KY_Beattyville_708136_1961_24000_geo', 'KY_Belfry_708145_1992_24000_geo', 'KY_Belfry_708147_1978_24000_geo', 'KY_Bernstadt_708158_1969_24000_geo', 'KY_Beverly_708161_1979_24000_geo', 'KY_Billows_708177_1952_24000_geo', 'KY_Blackey_708183_1954_24000_geo', 'KY_Blackwater_708188_1979_24000_geo', 'KY_Blaine_708191_1953_24000_geo', 'KY_Bledsoe_708195_1980_24000_geo', 'KY_Boltsfork_708203_1971_24000_geo', 'KY_Booneville_708207_1961_24000_geo', 'KY_Booneville_708209_1961_24000_geo', 'KY_Brushart_708255_1949_24000_geo', 'KY_Buckhorn_708264_1961_24000_geo', 'KY_Buckhorn_708265_1979_24000_geo', 'KY_Burnside_708284_1965_24000_geo', 'KY_Burnside_708285_1965_24000_geo', 'KY_Campt

In [4]:
#Make chips for each topo map
for cr in chip_n:
    #Set workspace
    arcpy.env.workspace = in_fold + in_toposF
    #Set local variables and make folders
    quadNm = cr.split("_")[1].replace(" ", "_")
    #os.mkdir(chips_outF + cr)
    subdir = chips_outF + cr + "/"
    out_folder= in_fold + "processing/ky_chips4/"
    #Read in topo
    inRaster = in_fold + in_toposF + cr  + ".tif"
    #Copy topo to 8-bit PNG
    inRaster2 = arcpy.CopyRaster_management(inRaster, to_8bitF + cr + ".png",
                                "", "",256,"NONE","NONE","8_BIT_UNSIGNED","NONE","NONE", "PNG", 
                                  "NONE")
    #Read in Mines
    in_mines = in_fold + in_minesF + cr  + ".shp"
    in_training =  in_fold + in_minesF + cr  + ".shp"
    #Add and populate class value field if it doesn't already exist
    if 'classvalue' not in [f.name for f in arcpy.ListFields(in_training)]:
        arcpy.AddField_management(in_training, "classvalue", "SHORT")
        arcpy.CalculateField_management(in_training, "classvalue", 1, "PYTHON3")
    #Define image chip parameters
    image_chip_format = "PNG"
    tile_size_x = "256"
    tile_size_y = "256"
    stride_x= "128"
    stride_y= "128"
    output_nofeature_tiles= "ONLY_TILES_WITH_FEATURES"
    metadata_format= "Classified_Tiles"
    start_index = 0
    classvalue_field = "classvalue"
    buffer_radius = 0
    in_mask_polygons = in_fold + in_quadsF + cr + ".shp"
    rotation_angle = 0
    reference_system = "MAP_SPACE"
    processing_mode = "PROCESS_AS_MOSAICKED_IMAGE"
    blacken_around_feature = "NO_BLACKEN"
    crop_mode = "FIXED_SIZE"

    #Create image chips
    ExportTrainingDataForDeepLearning(inRaster2, out_folder, in_training, 
                 image_chip_format,tile_size_x, tile_size_y, stride_x, 
                 stride_y,output_nofeature_tiles, metadata_format, start_index, 
                 classvalue_field, buffer_radius, in_mask_polygons, rotation_angle, 
                 reference_system, processing_mode, blacken_around_feature, crop_mode)

In [ ]:
inRaster = "c:/test/InputRaster.tif"
out_folder = "c:/test/OutputFolder"
in_training = "c:/test/TrainingData.shp"
image_chip_format = "TIFF"
tile_size_x = "256"
tile_size_y = "256"
stride_x= "128"
stride_y= "128"
output_nofeature_tiles= "ONLY_TILES_WITH_FEATURES"
metadata_format= "Labeled_Tiles"
start_index = 0
classvalue_field = "Classvalue"
buffer_radius = 0
in_mask_polygons = "MaskPolygon"
rotation_angle = 0
reference_system = "PIXEL_SPACE"
processing_mode = "PROCESS_AS_MOSAICKED_IMAGE"
blacken_around_feature = "NO_BLACKEN"
crop_mode = “FIXED_SIZE”